# INFO 7390 Advanced Data Science
## -- Assignment 1 
### Hao Wu 001443861

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [7]:
url = 'https://www.kaggle.com/lava18/google-play-store-apps/download'
r = requests.get(url)

r.headers

{'Date': 'Wed, 23 Sep 2020 20:18:55 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Cache-Control': 'no-cache, no-store', 'Pragma': 'no-cache', 'Content-Encoding': 'gzip', 'Set-Cookie': 'CSRF-TOKEN=CfDJ8LdUzqlsSWBPr4Ce3rb9VL-N4VKruYH8g-vjRiAQCBJNIoGrrdJ1V0tWIYJ22pBNd3t40YgIMyHHnEq9l8xQqQAOZHpv2zVzXWk55RgeBV1p3pRgQ0u0eL1lMibjB1AkE-tJKq52f59sXM8fMNH7gzk; path=/; secure; samesite=lax; httponly, XSRF-TOKEN=CfDJ8LdUzqlsSWBPr4Ce3rb9VL9KtuuiCDjGLQAbVWnkfDB9eCmn6L3Kf9QwB6ED4xysYBB71mDAhseo_NHCFBqyawBHuN01lum5O4uKKDLJ8TIgwkdif0dUMzDOcf6Prr4rGTt_rhh2U3D-fEFAXDbwTCQ; path=/; secure; samesite=lax, CLIENT-TOKEN=eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJrYWdnbGUiLCJhdWQiOiJjbGllbnQiLCJzdWIiOm51bGwsIm5idCI6IjIwMjAtMDktMjNUMjA6MTg6NTUuNzU1MjQ0NFoiLCJpYXQiOiIyMDIwLTA5LTIzVDIwOjE4OjU1Ljc1NTI0NDRaIiwianRpIjoiM2VjOGU1ZWQtMWM4OC00Nzc1LTg4MDMtYzFhNzUyNjY4N2NlIiwiZXhwIjoiMjAyMC0xMC0yM1QyMDoxODo1NS43NTUyNDQ0WiIsImFub24iOnRydWUsImZmIjpbIkZsZXhpYmxlR3B1IiwiS2VybmVsc0ludGVybmV0IiwiRGF0YUV4cGxvcmVyVjIiLCJ

In [5]:
data = pd.read_csv('https://www.kaggle.com/lava18/google-play-store-apps/download',encoding='ISO-8859-1')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2


In [25]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
